In [1]:
import sys
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python/")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/")
sys.path.append("/home/rossidaniel/Dropbox")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python")

In [2]:
import Bin_Packing_Py as BP
import BP_print
import BP_plot
import BP_utilities
import HA_routines
import BP_instances

import matplotlib
%matplotlib nbagg
matplotlib.rcParams['figure.figsize'] = [6,3.5]

from ortools.linear_solver import pywraplp
import random
import time
import csv

import pandas as pd
from mail import send_mail
from Test_BP_G import euristic
import multiprocessing

In [3]:
def tc_sfida(ist):
    item_list = []
    bin_list = []

    bin_list.append(BP.Bin(2.70, 100, 2.45))
    bin_list.append(BP.Bin(0, 0, 0))

    n_items =random.randint(3,10)

    for i in range(n_items):
        w=random.uniform(ist.width.a,ist.width.b)
        d=random.uniform(ist.depth.a,ist.depth.b)
        h=random.uniform(0.5,1.5)
        item = BP.Item(i,h,d,w)
        item.stackable = random.randint(0,0)
        item.order_id = random.randint(0,3)
        item.weight = 4
        item_list.append(item)

    return item_list, bin_list

In [4]:
class istanza:
    def __init__(self,int1,int2,ist_id):
        self.index=ist_id
        self.width=int1
        self.depth=int2
        
class intervallo:
    def __init__(self,a,b):
        self.a=a
        self.b=b

In [5]:
class model_bin_packing:
    def __init__(self,item_list,bin_list):
        self.bp = BP.BinPack()
        self.bp.items.extend(item_list)
        self.bp.bins.extend(bin_list)
        
    def buildModel(self,amount_time=300000):
        solver = pywraplp.Solver('Model2DRS', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        #importo dimensioni dei pacchi e del camion
        n=len(self.bp.items)

        w =[self.bp.items[i].width for i in range(n)]
        d =[self.bp.items[i].depth for i in range(n)]
        h =[self.bp.items[i].height for i in range(n)]
        
        o =[[True if self.bp.items[i].order_id < self.bp.items[j].order_id else False for j in range(n)]for i in range(n)]

        M=sum([max(w[i],d[i]) for i in range(n)])

        W =self.bp.bins[0].width
        D =solver.NumVar(0,M,"D")
        H =self.bp.bins[0].height
        
        Mw=W+M
        Md=self.bp.bins[0].depth+M
        Mz=H+max(h)
                
        # definisco le variabili
        l =[[solver.BoolVar("l%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        b =[[solver.BoolVar("b%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        r =[solver.BoolVar("r%d" % (i)) for i in range(n)]
        
        sx = [[solver.BoolVar("sx%d%d" % (i,j)) for i in range(n)]for j in range(n)]
        dx = [[solver.BoolVar("dx%d%d" % (i,j)) for i in range(n)]for j in range(n)]
        hg = [[solver.BoolVar("hg%d%d" % (i,j)) for i in range(n)]for j in range(n)]
        
        s1 =[solver.BoolVar("s1%d" % (i)) for i in range(n)]
        s2 =[solver.BoolVar("s2%d" % (i)) for i in range(n)]
        s3 =[solver.BoolVar("s3%d" % (i)) for i in range(n)]

        x =[solver.NumVar(0,solver.infinity(),"x%d" % i) for i in range(n)]
        y =[solver.NumVar(0,solver.infinity(),"y%d" % i) for i in range(n)]


        #definisco i constraints
        for i in range(n):
            for j in range(n):
                if(i < j):
                    solver.Add(l[i][j] + l[j][i] + b[i][j] + b[j][i] >= 1)                           #(1)
                if(i != j):
                    solver.Add(x[i] - x[j] + Mw * l[i][j] <= Mw - w[i]*(1-r[i]) - d[i]*r[i])         #(2)
                    solver.Add(y[i] - y[j] + Md * b[i][j] <= Md - d[i]*(1-r[i]) - w[i]*r[i])         #(3)
                    
                    solver.Add(sx[i][j] <= l[i][j])
                    solver.Add(sx[i][j] <= 1 - b[i][j] + b[j][i])
                    solver.Add(sx[i][j] >= l[i][j] - 10*(b[i][j] + b[j][i]))
                    
                    solver.Add(dx[i][j] <= l[j][i])
                    solver.Add(dx[i][j] <= 1 - b[i][j] + b[j][i])
                    solver.Add(dx[i][j] >= l[j][i] - 10*(b[i][j] + b[j][i]))
                    
                    solver.Add(hg[i][j] <= b[i][j])
                    solver.Add(hg[i][j] <= 1 - l[i][j] + l[j][i])
                    solver.Add(hg[i][j] >= b[i][j] - 10*(l[i][j] + l[j][i]))

                    solver.Add(sx[i][j]*o[i][j] <= s1[i])
                    solver.Add(dx[j][i]*o[i][j] <= s2[i])
                    solver.Add(hg[i][j]*o[i][j] <= s3[i])
                    
            solver.Add(x[i] + w[i]*(1-r[i]) + d[i]*r[i] <= W)                                        #(4)
            solver.Add(y[i] + d[i]*(1-r[i]) + w[i]*r[i] <= D)                                        #(5)
            solver.Add(s1[i] + s2[i] + s3[i] <= 2)
            
        #funzione obiettivo
        objective = solver.Objective()
        objective.SetCoefficient(D,1)
        objective.SetMinimization()

        #soluzione
        solver.SetTimeLimit(amount_time)
        status = solver.Solve()
        t= solver.wall_time()*0.001
        
        s = BP.Solution(self.bp)
        for i in range(len(self.bp.items)):
            rt = False if r[i].solution_value() == 0 else True
            s.pushItem(self.bp.items[i],0,BP.Point(x[i].solution_value(),y[i].solution_value(),0),rt,False)
        return s,t,n,status
    
    def BuildSolution(self,s):
        print s
    
    def plotSolution(self,s):
        BP_plot.plotSolution(s)
    
    def plot3Dsolution(self,s):
        BP_plot.plot3DSolution(s,0)

    def feasible(self):
        return self.bp.bins[0].height*self.bp.bins[0].width >= sum([self.bp.items[i].width*self.bp.items[i].height for i in range(len(self.bp.items))])

In [6]:
def calculate_result(istance,p_result):
    model = model_bin_packing(istance[0],istance[1])
    s,t,n,status=model.buildModel()
    s.computeObj()
    mod_obj =round(s.obj,2)
    eu_obj =round(euristic(istance[0],istance[1]),2)
    Time =round(t,3)
    n_items =n
    p_result[istance[3]]=[mod_obj,eu_obj,Time,n_items,istance[2],status]
    

In [7]:
ist = []
df = pd.read_excel('istanza.xlsx')
for i in range(len(df)):
    ist.append(istanza(intervallo(df.w_a[i],df.w_b[i]),intervallo(df.d_a[i],df.d_b[i]),i))

In [8]:
models_x_istance = 100
pre = []

for i in ist:
    for j in range(models_x_istance):
        items,bins=tc_sfida(i)
        pre.append([items,bins,i.index,((i.index*100)+j)])

In [9]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [10]:
pre =sorted(pre, key=lambda x: len(x[0]))
pre = list(chunks(pre, multiprocessing.cpu_count()))

In [11]:
tm=time.time()
manager = multiprocessing.Manager()
p_result = manager.dict()
for i in pre:
    jobs = []
    for j in i:
        p = multiprocessing.Process(target=calculate_result, args=(j,p_result))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
print time.time()-tm

16501.888262


In [12]:
result=p_result.values()

mod_obj = [n_it[0] for n_it in result]
eu_obj = [n_it[1] for n_it in result]
Time = [n_it[2] for n_it in result]
n_items = [n_it[3] for n_it in result]
istance = [n_it[4] for n_it in result]
optime = [n_it[5] for n_it in result]

In [13]:
tot_df =pd.DataFrame({'n_items': n_items, 'Eu_obj': eu_obj,'Mod_obj': mod_obj, 'Time': Time, 'Istance': istance, 'Optime': optime})

for i in range(len(ist)):
    result_ist =tot_df[(tot_df['Istance'] == i)]
    result_ist =result_ist.drop(['Istance'],axis=1)
    result_ist.to_excel('result'+str(i)+'.xlsx', encoding='utf8')

tot_df.drop(['Istance'],axis=1)
tot_df.to_excel('result_tot.xlsx', encoding='utf8')

In [14]:
tot_df

,Eu_obj,Istance,Mod_obj,Optime,Time,n_items
0,5.07,0,5.07,0,0.026,3
1,6.00,0,6.00,0,11.879,6
2,9.46,0,8.70,1,332.470,9
3,5.45,0,5.31,0,1.054,5
4,10.52,0,10.49,1,302.080,10
5,6.72,0,6.72,0,1.616,5
6,6.55,0,6.10,0,225.496,8
7,4.94,0,4.59,0,38.926,6
8,10.11,0,9.42,1,338.107,9
9,4.38,0,4.38,0,2.433,5


In [15]:
send_mail()